<a href="https://colab.research.google.com/github/Vamarinpe/PAE/blob/master/CodRecurso.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Por agente

# importa la librería
import pandas as pd
import numpy as np
pd.set_option('display.notebook_repr_html', False)
import matplotlib.pyplot as plt
%matplotlib inline
from datetime import datetime
import csv

Datos = pd.read_csv(
    "https://raw.githubusercontent.com/Vamarinpe/PAE/master/Recurso.csv",
    sep = ',',         # separador de campos
    thousands = None,  # separador de miles para números
    decimal = '.',
    index_col=["Fecha","Codigo Agente","Recurso"],
    usecols=["Fecha","Codigo Agente","Recurso","Precio Oferta Ideal"],      # Datosnuevo=Datos[['Fecha','Codigo Agente','Recurso']]  - otra forma
    parse_dates=["Fecha"],
    dayfirst=True)


#Datos = pd.read_csv(
    #"https://raw.githubusercontent.com/Vamarinpe/Ensayo2/master/Datos.csv",
    #sep = ',',         # separador de campos
    #thousands = None,  # separador de miles para números
    #decimal = '.',
    #quoting=csv.QUOTE_NONNUMERIC)

#Datos['Fecha'] = Datos.Fecha\
                             #.map(lambda x : datetime.strptime(x, '%Y-%m-%d'))

In [0]:
# verificar lectura
len(Datos)
Datos.columns
Datos.describe()
Datos.head(60)
Datos.count()

In [0]:
# contando registros con groupby
Datos.groupby('Codigo Agente')['Precio Oferta Ideal'].count()
Datos.groupby('Recurso')['Precio Oferta Ideal'].count()
Datos.groupby('Fecha')['Precio Oferta Ideal'].count()            # también sin el ['Precio Oferta Ideal']
#Datos.groupby('Codigo Agente').count()

In [0]:
# muestra con loc, debe decir los indices especificos, de mayor a menor
Datos.loc[('2012-09-04')]
Datos.loc[('2012-09-04','EPMG')]
plt.plot(Datos.loc[('2017-09-05','EPMG')])
plt.plot(Datos.loc[('2017-09-06','EPMG')])
plt.plot(Datos.loc[('2019-08-31','EPMG')])
plt.show()
#Datos.loc['2012-01-01':'2013-12-31']                         # también funciona para rangos

In [0]:
# muestra con iloc, encuentra el dato que le digo en numero
Datos.iloc[-2:]

In [0]:
# muestra con xs, filtra y muestra todo lo del nivel especifico indicado, muestra lo de los demás indices
Datos.xs('2012-06-08',level='Fecha')
Datos.xs('EPMG',level='Codigo Agente')
Datos.xs('GUATAPE',level='Recurso').plot()
Datos.xs('PLAYAS',level='Recurso').plot()
plt.rcParams["figure.figsize"] = (20,8)


In [0]:
# funcion para obtener los recursos de más capacidad en cada año (10 primeros)
def capacidad(x):
  Cap = pd.read_csv(
    x,
    sep = ';',         # separador de campos
    thousands = None,  # separador de miles para números
    decimal = '.',
    index_col=["Recurso"],
    usecols=["Recurso","Capacidad Efectiva Neta kW","Combustible por defecto"])
  return Cap.sort_values(by='Capacidad Efectiva Neta kW',ascending=False).drop_duplicates().head(10)

C2000=capacidad("/content/Capacidad_Efectiva_Neta_(kW)_2000.csv")
#C2001=capacidad("/content/Capacidad_Efectiva_Neta_(kW)_2001.csv")

C2000

In [0]:
# cuenta de esos primeros, que combustible usan
C2000.groupby('Combustible por defecto').count()
C2000['Combustible por defecto']                # muestra la columna
Datos.index.names[2]                            # muestra nombre index 2
Datos.index.get_level_values('Recurso')         # muestra los datos de un indice

In [0]:
# filtra Datos y muestra solo los que tienen mayor capacidad, agrega columna con el combustible
Datos['Combustible']=Datos.index.get_level_values('Recurso').map(C2000['Combustible por defecto'])
NuevaDatos=Datos[Datos.index.get_level_values('Recurso').isin(list(C2000.index))]
NuevaDatos